In [3]:
import os
import sys
import json
sys.path.append("../../../")
current_file_dir = os.getcwd()


from prompt_scope.core.llms.dashscope_llm import DashscopeLLM
from prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer import StaticTipsOptimizer

# Step1: Load data

In [2]:
with open("../../data/college_medical_statistics/sample_train_data.jsonl", "r") as f:
    train_set = []
    for line in f:
        train_set.append(json.loads(line))

with open("../../data/college_medical_statistics/sample_test_data.jsonl", "r") as f:
    test_set = []
    for line in f:
        test_set.append(json.loads(line))

with open("../../data/college_medical_statistics/init_system_prompt.txt", "r") as f:
    init_system_prompt = f.read().strip()

# Step2: Define a scoring function

In [4]:
def is_good_case(prediction, ground_truth):
    prediction = prediction.split("<answer>")[-1].split("</answer>")[0].strip()
    ground_truth = ground_truth
    if ground_truth in prediction:
        return True, 1 # Good case, score
    else:
        return False, 0 # Bad case, score

# Step3: Run optimizer

In [3]:
infer_llm = DashscopeLLM(model="qwen-plus", temperature=0.0)
optim_llm = DashscopeLLM(model="qwen-max", temperature=0.0)

In [4]:
stg = StaticTipsOptimizer(
    init_system_prompt=init_system_prompt,
    infer_llm=infer_llm,
    optim_llm=optim_llm,
    train_set=train_set,
    test_set=test_set,
    is_good_case_func=is_good_case,
    details_save_dir=f"{current_file_dir}/details_result",
    save_steps = 30,
    epoch = 10
)
optimized_prompt = stg.run()

2025-01-15 10:50:03.078 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:_before_run:79 - 
============== evaluating initial system prompt on test ===============
Score: 100%|██████████████████████████████████| 53/53 [00:00<00:00, 6241.87it/s]
2025-01-15 10:50:33.074 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:_before_run:86 - test_avg_score: 
0.7547169811320755
2025-01-15 10:55:03.412 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:123 - 
=====Epoch-1 Step-30/530=====
2025-01-15 10:55:03.414 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:124 - 
=====Epoch-1/10 Step-30/530 updated tips=====
None
2025-01-15 10:55:03.415 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:129 - 
=====Epoch-1/10 Step-30/530 evaluate updated tips=====


Predict: 100%|██████████████████████████████████| 53/53 [00:26<00:00,  1.98it/s]

Score: 100%|█

- 当进行样本率 \( p \) 与总体率 \( \pi_0 \) 的比较时，建议关注总体率 \( \pi_0 \) 是否接近 0.5，不建议仅关注样本率 \( p \)。
- 建议确保样本含量 \( n \) 足够大，并且 \( np \) 和 \( n(1-p) \) 都大于等于 5，以保证正态近似的合理性。


Epoch-1/10 Training: 100%|██████████████████████| 53/53 [06:57<00:00,  7.88s/it]
2025-01-15 10:57:56.853 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:123 - 
=====Epoch-2 Step-60/530=====
2025-01-15 10:57:56.854 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:124 - 
=====Epoch-2/10 Step-60/530 updated tips=====
- 当进行样本率 \( p \) 与总体率 \( \pi_0 \) 的比较时，建议关注总体率 \( \pi_0 \) 是否接近 0.5，不建议仅关注样本率 \( p \)。
- 建议确保样本含量 \( n \) 足够大，并且 \( np \) 和 \( n(1-p) \) 都大于等于 5，以保证正态近似的合理性。
2025-01-15 10:57:56.855 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:129 - 
=====Epoch-2/10 Step-60/530 evaluate updated tips=====


Predict: 100%|██████████████████████████████████| 53/53 [00:22<00:00,  2.40it/s]

Score: 100%|█████████████████████████████████| 53/53 [00:00<00:00, 19406.21it/s]
2025-01-15 10:58:18.964 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:137 - 
test_avg_scor

- 当进行简单线性回归分析时，建议认识到 \( r \) 和 \( b \) 的假设检验是等价的，不建议将它们视为独立的。
- 当 \( r \) 有统计学意义时，可以推断 \( b \) 也有统计学意义，不建议认为 \( b \) 的统计学意义不确定。


2025-01-15 11:01:22.214 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:123 - 
=====Epoch-2 Step-90/530=====
2025-01-15 11:01:22.215 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:124 - 
=====Epoch-2/10 Step-90/530 updated tips=====
- 当进行样本率 \( p \) 与总体率 \( \pi_0 \) 的比较时，建议关注总体率 \( \pi_0 \) 是否接近 0.5，不建议仅关注样本率 \( p \)。
- 建议确保样本含量 \( n \) 足够大，并且 \( np \) 和 \( n(1-p) \) 都大于等于 5，以保证正态近似的合理性。
- 当进行简单线性回归分析时，建议认识到 \( r \) 和 \( b \) 的假设检验是等价的，不建议将它们视为独立的。
- 当 \( r \) 有统计学意义时，可以推断 \( b \) 也有统计学意义，不建议认为 \( b \) 的统计学意义不确定。
2025-01-15 11:01:22.215 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:129 - 
=====Epoch-2/10 Step-90/530 evaluate updated tips=====


Predict: 100%|██████████████████████████████████| 53/53 [00:18<00:00,  2.85it/s]

Score: 100%|██████████████████████████████████| 53/53 [00:00<00:00, 5829.39it/s]
2025-01-15 11:01:40.822 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer

...

2025-01-15 11:38:54.797 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:123 - 
=====Epoch-9 Step-450/530=====
2025-01-15 11:38:54.799 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:124 - 
=====Epoch-9/10 Step-450/530 updated tips=====
- 当进行样本率 \( p \) 与总体率 \( \pi_0 \) 的比较时，建议关注总体率 \( \pi_0 \) 是否接近 0.5，不建议仅关注样本率 \( p \)。
- 建议确保样本含量 \( n \) 足够大，并且 \( np \) 和 \( n(1-p) \) 都大于等于 5，以保证正态近似的合理性。
- 当进行简单线性回归分析时，建议认识到 \( r \) 和 \( b \) 的假设检验是等价的，不建议将它们视为独立的。
- 当 \( r \) 有统计学意义时，可以推断 \( b \) 也有统计学意义，不建议认为 \( b \) 的统计学意义不确定。
- 当对简单线性回归模型进行显著性检验的结果是不能拒绝 \( H_0 \) 时，建议认识到这并不意味着模型无应用价值，不建议直接断定模型无效。
- 建议理解不能拒绝 \( H_0 \) 只是表明没有足够的证据支持 \( X \) 和 \( Y \) 之间存在显著线性关系，而不代表两者之间完全没有关系。
- 当处理统计显著性问题时，建议综合考虑实际背景和数据特征，不建议仅依赖于统计检验结果来判断模型的应用价值。
- 当遇到统计假设检验的问题时，建议首先明确题目要求验证的是无效假设\(H_0\)还是备择假设\(H_1\)，不建议直接套用常见的假设形式而不考虑题意。
- 在阅读题目时，建议仔细审题，特别是注意题干中给出的具体条件或特殊说明，不建议忽略这些细节。
- 当考虑使用近似正态\(u\)检验比较样本率\(p\)与总体率\(\pi_0\)时，建议重点关注总体率\(\pi_0\)是否接近0.5以及样本容量\(n\

- 当考虑使用近似正态\(\mathrm{u}\)检验时，建议依据总体率\(\pi_0\)而非样本率\(p\)来判断，不建议仅凭样本数据特征做出决策。
- 在评估模型或进行统计测试时，建议综合考虑理论基础和实际数据特性，不建议孤立地依赖单一指标或假设。


Epoch-9/10 Training: 100%|██████████████████████| 53/53 [05:18<00:00,  6.01s/it]
2025-01-15 11:41:03.422 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:123 - 
=====Epoch-10 Step-480/530=====
2025-01-15 11:41:03.424 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:124 - 
=====Epoch-10/10 Step-480/530 updated tips=====
- 当进行样本率 \( p \) 与总体率 \( \pi_0 \) 的比较时，建议关注总体率 \( \pi_0 \) 是否接近 0.5，不建议仅关注样本率 \( p \)。
- 建议确保样本含量 \( n \) 足够大，并且 \( np \) 和 \( n(1-p) \) 都大于等于 5，以保证正态近似的合理性。
- 当进行简单线性回归分析时，建议认识到 \( r \) 和 \( b \) 的假设检验是等价的，不建议将它们视为独立的。
- 当 \( r \) 有统计学意义时，可以推断 \( b \) 也有统计学意义，不建议认为 \( b \) 的统计学意义不确定。
- 当对简单线性回归模型进行显著性检验的结果是不能拒绝 \( H_0 \) 时，建议认识到这并不意味着模型无应用价值，不建议直接断定模型无效。
- 建议理解不能拒绝 \( H_0 \) 只是表明没有足够的证据支持 \( X \) 和 \( Y \) 之间存在显著线性关系，而不代表两者之间完全没有关系。
- 当处理统计显著性问题时，建议综合考虑实际背景和数据特征，不建议仅依赖于统计检验结果来判断模型的应用价值。
- 当遇到统计假设检验的问题时，建议首先明确题目要求验证的是无效假设\(H_0\)还是备择假设\(H_1\)，不建议直接套用常见的假设形式而不考虑题意。
- 在阅读题目时，建议仔细审题，特别是注意题干中给出的具体条件或特殊说明，不建议忽略这

- 当评估是否可以使用近似正态\(\mathrm{u}\)检验时，建议关注总体率\(\pi_0\)而非样本率\(p\)，不建议仅依据样本数据特征做出决定。
- 在考虑模型或统计方法的应用条件时，建议仔细阅读题目要求，确保理解所讨论的具体参数或指标，不建议直接套用一般规则而忽略具体情况。


2025-01-15 11:45:48.124 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:123 - 
=====Epoch-10 Step-530/530=====
2025-01-15 11:45:48.126 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:124 - 
=====Epoch-10/10 Step-530/530 updated tips=====
- 当进行样本率 \( p \) 与总体率 \( \pi_0 \) 的比较时，建议关注总体率 \( \pi_0 \) 是否接近 0.5，不建议仅关注样本率 \( p \)。
- 建议确保样本含量 \( n \) 足够大，并且 \( np \) 和 \( n(1-p) \) 都大于等于 5，以保证正态近似的合理性。
- 当进行简单线性回归分析时，建议认识到 \( r \) 和 \( b \) 的假设检验是等价的，不建议将它们视为独立的。
- 当 \( r \) 有统计学意义时，可以推断 \( b \) 也有统计学意义，不建议认为 \( b \) 的统计学意义不确定。
- 当对简单线性回归模型进行显著性检验的结果是不能拒绝 \( H_0 \) 时，建议认识到这并不意味着模型无应用价值，不建议直接断定模型无效。
- 建议理解不能拒绝 \( H_0 \) 只是表明没有足够的证据支持 \( X \) 和 \( Y \) 之间存在显著线性关系，而不代表两者之间完全没有关系。
- 当处理统计显著性问题时，建议综合考虑实际背景和数据特征，不建议仅依赖于统计检验结果来判断模型的应用价值。
- 当遇到统计假设检验的问题时，建议首先明确题目要求验证的是无效假设\(H_0\)还是备择假设\(H_1\)，不建议直接套用常见的假设形式而不考虑题意。
- 在阅读题目时，建议仔细审题，特别是注意题干中给出的具体条件或特殊说明，不建议忽略这些细节。
- 当考虑使用近似正态\(u\)检验比较样本率\(p\)与总体率\(\pi_0\)时，建议重点关注总体率\(\pi_0\)是否接近0.5以及样本容量\(